# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date;title;venue;excerpt;citation;url_slug;paper_url
2016-01-01;Mathematical foundations of antibody aﬃnity maturation;Hal;PhD Thesis manuscript.;Balelli, I. (2016). Mathematical foundations of antibody aﬃnity maturation (Doctoral dissertation, Université Sorbonne Paris Cité).;2016-1;http://irenebalelli.github.io/files/papapers/Irene_Balelli_these.pdf
2018-01-01;Branching random walks on binary strings for evolutionary processes in adaptive immunity;arXiv;In this article, we study branching random walks on graphs modeling division-mutation processes inspired by adaptive immunity. We apply the theory of expander graphs on mutation rules in evolutionary processes and obtain estimates for the cover times of the branching random walk.;Balelli, I., Milišić, V., & Wainrib, G. (2018). Random walks on binary strings applied to the somatic hypermutation of B-cells. <i>Mathematical biosciences<i>, 300, 168-186. ;	2018-2 ; https://arxiv.org/pdf/1607.00927v1.pdf
2018-01-01;Random walks on bina

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [4]:
publications = pd.read_csv("publications.tsv", sep=";", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2016-01-01,Mathematical foundations of antibody aﬃnity ma...,Hal,PhD Thesis manuscript.,"Balelli, I. (2016). Mathematical foundations o...",2016-1,http://irenebalelli.github.io/files/papapers/I...
1,2018-01-01,Branching random walks on binary strings for e...,arXiv,"In this article, we study branching random wal...","Balelli, I., Milišić, V., & Wainrib, G. (2018)...",\t2018-2,https://arxiv.org/pdf/1607.00927v1.pdf
2,2018-01-01,Random walks on binary strings applied to the ...,Mathematical biosciences,This paper establish a mathematical framework ...,"Balelli, I., Milišić, V., & Wainrib, G. (2018)...",2018-1,http://irenebalelli.github.io/files/papapers/r...
3,2019-01-01,Multi-type Galton-Watson processes with affini...,Bulletin of mathematical biology,"Inspired by antibody affinity maturation, the ...","Balelli, I., Milišić, V., & Wainrib, G. (2019)...",2019-1,http://irenebalelli.github.io/files/papapers/m...
4,2019-01-01,Dynamics of the humoral immune response to a p...,Journal of virology,In this paper data from three phase 1 trials o...,"Pasin, C., Balelli, I., Van Effelterre, T., et...",2019-2,https://journals.asm.org/doi/pdf/10.1128/jvi.0...
5,2019-01-01,Viral rebound kinetics following single and co...,BioRxiv,In this paper we conduct a detailed analysis o...,"Prague, M., Gerold, J. M., Balelli, I., et al....",2019-3,https://www.biorxiv.org/content/10.1101/700401v2
6,2020-01-01,"A model for establishment, maintenance and rea...",Journal of theoretical biology,This article describes the biological motivati...,"Balelli, I., Pasin, C., Prague, M., et al. (20...",2020-1,https://www.sciencedirect.com/science/article/...
7,2021-01-01,Parameter estimation in nonlinear mixed effect...,arXiv,We present a parameter estimation method for n...,"Clairon, Q., Pasin, C., Balelli, I., et al. (2...",2021-2,https://arxiv.org/pdf/2102.11543.pdf
8,2021-01-01,A Probabilistic Framework for Modeling the Var...,Lecture Notes in Computer Science,In this paper we propose a new Bayesian framew...,"Balelli I., Silva S., Lorenzi M. (2021). A Pro...",2021-1,https://hal.archives-ouvertes.fr/hal-03152886/
9,2021-01-01,A Differentially Private Probabilistic Framewo...,Submitted to Journal of Machine Learning for B...,This paper is an extension of the work present...,"Balelli I., Silva S., Lorenzi M. (2021). A Dif...",2021-3,http://irenebalelli.github.io/files/papapers/B...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [20]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [21]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [22]:
!ls ../_publications/

2016-01-01-2016-1.md   2019-01-01-2019-2.md   2021-01-01-2020-2.md
2016_1.md              2019-01-01-2019-3.md   2021-01-01-2021-1.md
2018-01-01-?2018-2 .md 2019_1.md              2021-01-01-2021-3.md
2018-01-01-2018-1.md   2019_2.md              2021_1.md
2018_1.md              2019_3.md              2021_2.md
2018_2.md              2020-01-01-2020-1.md   2021_3.md
2019-01-01-2019-1.md   2020_1.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).